In [67]:
import numpy as np
import os
import subprocess 
import MDAnalysis as md
from MDAnalysis.analysis.leaflet import LeafletFinder
import re

# Notebook for constructing oil compartment

## Requirements
### - Define Paths bellow;

In [68]:
IL_name_NP = 'MC3H'
IL_name_N1 = 'MC3'
sterol_name = 'CHOL'
vesicle_radius = 25 # nm
no_N1_lipids = 14000 # number of ILs - can be adapted
no_CHOL_lipids = 7000 # number of Cholesterols - can be adapted

In [69]:
main_dir = '.'
mdp_loc_BL = 'MDPs/'
ILs_itp = 'ITP/MC3_KC2_DP_DT/_LI5_LI2_LI10_BMHB.itp'

PCG_path = '' #Define path to PCG/TS2CG instalation

RNA = 'PDBs/ssDNA.gro'
RNA_itp = 'ITP/ssDNA.itp'

    
itpDir   = "ITP/"
IL_itp   = 'ITP/MC3_KC2_DP_DT_LI5_LI2_LI10_BMHB.itp'
Sterols  = "ITP/martini_v3.0_sterols_v1.0.itp"    
ffbonded = "ITP/martini_v3.0_ffbonded.itp"

####  functions

In [91]:
def get_radius (x,y,z):
    # Calculate the centroid of the cylinder (center axis)
    centroid_x = np.mean(x)
    centroid_y = np.mean(y)
    centroid_z = np.mean(z)

    # Calculate the distances from the centroid to each point
    distances = np.sqrt((x - centroid_x)**2 + (y - centroid_y)**2 + (z - centroid_z)**2)

    # Find the minimum distance, which represents the internal radius of the cylinder
    internal_radius = np.min(distances)

    return internal_radius

def write_packmol_input(channel_radius, noW=1, noCL=1):
    f = open('solvate_channel.inp', 'w') 
    f.write(f'tolerance 3.0\nfiletype pdb\n\n')

    f.write(f'structure PDBs/MC3.pdb\n')
    f.write(f'  number {no_N1_lipids}\n') 
    f.write(f'  inside sphere 0. 0. 0. {int(channel_radius)}.\n')
    f.write(f'end structure\n')

    f.write(f'structure PDBs/CHOL.pdb\n')
    f.write(f'  number {no_CHOL_lipids}\n') 
    f.write(f'  inside sphere 0. 0. 0. {int(channel_radius)}.\n')
    f.write(f'end structure\n')
            
    f.write('output solvate_channel_rna2.pdb\n')
    f.close()
    return

def extract_lines(input_file, pattern):
    extracted_lines = []
    pattern_found = False

    with open(input_file, 'r') as f:
        for line in f:
            if pattern_found:
                extracted_lines.append(line)

            if pattern in line:
                pattern_found = True

    return extracted_lines

def write_topol (topol, noW, noCL):
    '''Writes out a topol.top file with the solvate channel times 4 including the correct itp files'''
    extracted_lines = extract_lines(topol, 'lower monolayer')


    with open('topol2.top', 'w') as topout:
        topout.write(f'#include "{itpDir}/martini_v3.0.0.itp"\n#include "{ffbonded}"\n#include "{RNA_itp}"\n\
#include "{itpDir}/martini_v3.0.0_phospholipids_v1.itp"\
\n#include "{IL_itp}"\n#include "{Sterols}"\n#include "{itpDir}/martini_v3.0.0_solvents_v1.itp"\n\
#include "{itpDir}/martini_v3.0.0_ions_v1.itp"\n')
        topout.write(f'[ system ]\n')
        topout.write('Compartment_Oil\n')
        topout.write(f'\n')
        topout.write(f'[ molecules ]\n')
        for i in range(1):
                topout.write(f'MC3    {no_N1_lipids}\n')
                topout.write(f'CHOL    {no_CHOL_lipids}\n')

    return

## Create oil compartment

In [96]:
write_packmol_input((vesicle_radius*100)-5, noW=0, noCL=0)

##### Run packmol

In [97]:
!/projects/DAMM/packmol-20.14.3/packmol < solvate_channel.inp


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 20.14.3 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Types of coordinate files specified: pdb
  Seed for random number generator:      1234567
  Output file: solvate_channel_rna2.pdb
  Reading coordinate file: PDBs/MC3.pdb
  Reading coordinate file: PDBs/CHOL.pdb
  Number of independent structures:            2
  The structures are: 
  Structure            1 :PDBs/MC3.pdb(          12  atoms)
  Structure            2 :PDBs/CHOL.pdb(           9  atoms)
  Maximum number of GENCAN loops for all molecule packing:          400
 

##### Create box

In [98]:
os.system('gmx editconf -f solvate_channel_rna2.pdb -o channel_solvated_combined2.gro -box 70 70 70') 

                     :-) GROMACS - gmx editconf, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/LNP_builder/Building_material/Bleb_particle
Command line:
  gmx editconf -f solvate_channel_rna2.pdb -o channel_solvated_combined2.gro -box 70 70 70


Back Off! I just backed up channel_solvated_combined2.gro to ./#channel_solvated_combined2.gro.2#


Note that major changes are planned in future for editconf, to improve usability and utility.
Read 231000 atoms
No velocities found
    system size : 48.258 48.627 48.693 (nm)
    center      : -0.063  0.011 -0.055 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       : 35.063 34.989 35.055 (nm)
new center      : 35.000 35.000 35.000 (nm)
new box vectors : 70.000 70.000 70.000 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :343000.00               (nm^3)



GROMACS reminds you: "I'd be Safe and Warm if I was in L.A." (The Mamas and the Papas)



0

##### Write topol.top

In [99]:
write_topol('system2.top', noW=0, noCL=0)

### Min and relax

In [100]:
os.system(f"gmx grompp -f {mdp_loc_BL}/min.mdp -c channel_solvated_combined2.gro -p topol2.top -o m2.tpr -maxwarn 1")

                      :-) GROMACS - gmx grompp, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/LNP_builder/Building_material/Bleb_particle
Command line:
  gmx grompp -f MDPs//min.mdp -c channel_solvated_combined2.gro -p topol2.top -o m2.tpr -maxwarn 1

Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file MDPs//min.mdp]:
  Setting tcoupl from 'V-rescale' to 'no'. Temperature coupling does not
  apply to steep.


NOTE 2 [file MDPs//min.mdp]:
  Setting pcoupl from 'C-rescale' to 'no'. Pressure coupling does not apply
  to steep.



Setting the LD random seed to 2032820143

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'MC3'

Excluding 1 bonded neighbours molecule type 'CHOL'

Cleaning up constraints and constant bonded interactions with virtual sites
Analysing residue names:
There are: 21000      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

This run will generate roughly 30 Mb of data


Number of degrees of freedom in T-Coupling group System is 566997.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

There were 2 NOTEs

Back Off! I just backed up m2.tpr to ./#m2.tpr.2#

GROMACS reminds you: "Fresh Air, Green Hair" (Frank Black)



0

In [101]:
os.system(f"gmx mdrun -v -deffnm m2 -ntomp 5 -ntmpi 2")

                      :-) GROMACS - gmx mdrun, 2023.3 (-:

Executable:   /projects/DAMM/gromacs/avx2256/gromacs-2023.3/bin/gmx
Data prefix:  /projects/DAMM/gromacs/avx2256/gromacs-2023.3
Working dir:  /DAMM/hosts/damm085/data1/mvalerio/Lisbeth/LNP_builder/Building_material/Bleb_particle
Command line:
  gmx mdrun -v -deffnm m2 -ntomp 5 -ntmpi 2


Back Off! I just backed up m2.log to ./#m2.log.3#
Reading file m2.tpr, VERSION 2023.3 (single precision)
Update groups can not be used for this system because an incompatible virtual site type is used

On host ibcp22-085 1 GPU selected for this run.
Mapping of GPU IDs to the 2 GPU tasks in the 2 ranks on this node:
  PP:0,PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 2 MPI threads
Using 5 OpenMP threads per tMPI thread


NOTE: The number of threads is not equal to the number of (logical) cpus
      and the -pin option is set to auto: will not pin threads

0

In [102]:
### Create the index file
gro = 'm2.gro'
u = md.Universe(gro)
lipidsagg=u.select_atoms(f'resname {sterol_name} {IL_name_N1} {IL_name_NP} MC3H LI2H SITO CHOL A')
lipidsagg.write("index2.ndx", mode="w", name= 'Particle')
u.atoms.write("index2.ndx", mode="a", name= 'System')
